# Working with languages other than English

After reading this section, you should:

 - know how to download and use language models in Stanza, a Python library for processing many languages
 - how to interface Stanza with the spaCy natural language processing library

## Introduction

Part II introduced basic natural language processing tasks using examples written in the English language.

As a global *lingua franca*, English is a highly-resourced language in terms of natural language processing. Compared to many other languages, the amount of raw and human-annotated data available for English is far greater, while also covering a wider range of domains. In other words, it is easier to find large datasets for English that can be used to train models to perform diverse natural language processing tasks. 

Unfortunately, the imbalance in resources and research effort has led to a situation where the advances in processing the English language are occasionally claimed to hold for natural language in general. 

However, as Bender ([2019](https://thegradient.pub/the-benderrule-on-naming-the-languages-we-study-and-why-it-matters/)) has shown, *English is not a synonym for natural language*: even if one demonstrates that computers can achieve or surpass human-level performance in some natural language processing task for the English language, this does not mean that one has solved this task or problem for *natural language as a whole*.

To measure progress in the field of natural language processing and to ensure that as many languages as possible can benefit from language technology, it is highly desirable to conduct research on processing languages used across the world. 

## Stanza – a Python library for processing many languages

To get started with working languages other than English, we can use the Stanza library.

[Stanza](https://stanfordnlp.github.io/stanza/) is a Python library for natural language processing that provides pre-trained language models for [many languages](https://stanfordnlp.github.io/stanza/available_models.html) (Qi et al. [2020](https://www.aclweb.org/anthology/2020.acl-demos.14/)).

Stanza language models are trained on corpora annotated using the [Universal Dependencies](https://universaldependencies.org/) formalism, which means that these models can perform tasks such as tokenization, part-of-speech tagging, morphological tagging and dependency parsing. 

These are essentially the same tasks that we explored using the spaCy library in [Part II](../part_ii/03_basic_nlp.ipynb).

Let's start exploring Stanza by importing the library.

In [1]:
# Import the Stanza library
import stanza

To process a given language, we must first download a Stanza language model using the `download()` function.

The `download()` function requires a single argument, `lang`, which defines the language model to be downloaded.

To download a language model for a given language, retrieve the two-letter language code (e.g. *wo*) for the language from [the list of available language models](https://stanfordnlp.github.io/stanza/available_models.html) and pass the language code as a string object to the `lang` argument.

For example, the following code would download a model for Wolof, a language spoken in West Africa that belongs to the family of Niger-Congo languages. The model has been trained using the Wolof treebank (Dione [2019](https://www.aclweb.org/anthology/W19-8003/)).

```python
stanza.download(lang='wo')
```

For some languages, Stanza provides models that have been trained on different datasets. Within Stanza, these are referred to as *packages*. By default, Stanza automatically downloads the model trained on the largest dataset available for the language in question.

To select a model trained on a specific dataset, pass the name of the package as a string object to the `package` argument.

To exemplify, the following command downloads a model for Finnish trained on the [*FinnTreeBank*](https://universaldependencies.org/treebanks/fi_ftb/index.html) dataset instead of the default model, which is trained on the [*Turku Dependency Treebank*](https://universaldependencies.org/treebanks/fi_tdt/index.html) dataset.

```python
stanza.download(lang='fi', package='ftb')
```

The package names are provided in [the list of language models](https://stanfordnlp.github.io/stanza/available_models.html) available for Stanza.

To install the language model into the permanent storage on [CSC Notebooks](https://notebooks.csc.fi/), we must also pass the optional `model_dir` argument that points towards a directory in the permanent storage to the `download()` function.

Without using the permanent storage, the models are deleted when the server is shut down.

Run the cell below to download the Stanza language model for Wolof into the directory `../stanza_models`.

Note that `..` moves up one step in the directory structure relative to this notebook, which places the model into the directory `stanza_models` under the directory `notebooks`.

In [2]:
# Download a Stanza language model for Wolof into the directory "../stanza_models"
stanza.download(lang="wo", model_dir="../stanza_models")

2021-03-08 20:12:53 INFO: Downloading default packages for language: wo (Wolof)...
2021-03-08 20:12:53 INFO: File exists: ../stanza_models/wo/default.zip.
2021-03-08 20:12:54 INFO: Finished downloading models and saved to ../stanza_models.


### Loading a language model into Stanza

To load a Stanza language model into Python, we must first create a *Pipeline* object using the `Pipeline()` function available in the `stanza` module.

To exemplify the procedure, let's initialise a pipeline with a language model for Wolof.

To load a language model for Wolof into the pipeline, provide the string `wo` to the `lang` argument.

Because we did **not** place the language model into the default directory, we must also provide a string containing the path to the directory with Stanza language models to the `dir` argument.

We then store the resulting pipeline under the variable `nlp_wo`.

### Quick exercise

Check [the list of language models](https://stanfordnlp.github.io/stanza/available_models.html) available for Stanza and download a model for a language that you would like to work with. 

In [3]:
# Write your code below this line and press Shift and Enter to run the code


In [4]:
# Use the Pipeline() method to initialise a Stanza pipeline with a language model for Wolof, which
# is assigned to the variable 'nlp_wo'.
nlp_wo = stanza.Pipeline(lang='wo', dir="../stanza_models")

# Call the variable to examine the output
nlp_wo

2021-03-08 20:13:11 INFO: Loading these models for language: wo (Wolof):
| Processor | Package |
-----------------------
| tokenize  | wtb     |
| mwt       | wtb     |
| pos       | wtb     |
| lemma     | wtb     |
| depparse  | wtb     |

2021-03-08 20:13:11 INFO: Use device: cpu
2021-03-08 20:13:11 INFO: Loading: tokenize
2021-03-08 20:13:11 INFO: Loading: mwt
2021-03-08 20:13:11 INFO: Loading: pos
2021-03-08 20:13:11 INFO: Loading: lemma
2021-03-08 20:13:11 INFO: Loading: depparse
2021-03-08 20:13:12 INFO: Done loading processors!


Loading a language model into Stanza returns *Pipeline* object, which consists of a number of *processors* that use the language model to perform various natural language processing tasks.

The output above lists the processors under the heading of the same name, together with the packages used to train these processors.

As we learned in [Part II](http://localhost:8888/notebooks/part_ii/04_basic_nlp_continued.ipynb#Modifying-spaCy-pipelines), you might not always need all types of linguistic annotations. 

To speed up processing, you can define the processors to be included in the *Pipeline* object by providing the argument `processors` with a string object that contains the [processor names](https://stanfordnlp.github.io/stanza/pipeline.html#processors) separated by commas.

For example, using the following command to create a *Pipeline* would only include the processors for tokenization and part-of-speech tagging.

```python
nlp_wo = stanza.Pipeline(lang='wo', dir="../stanza_models", processors='tokenize, pos')
```

### Processing text using Stanza

Now that we have initialised a Stanza *Pipeline* with a language model, we can feed some text in Wolof to the model under `nlp` as a string object and store the result under the variable `doc_wo`.

In [5]:
# Feed text to the model under 'nlp_wo'; store result under the variable 'doc'
doc_wo = nlp_wo("Réew maa ngi lebe turam wi ci dex gi ko peek ci penku ak bëj-gànnaar, te ab balluwaayam bawoo ca Fuuta Jallon ca Ginne, di Dexug Senegaal. Ab kilimaam bu gëwéel la te di bu fendi te yor ñaari jamono: jamonoy nawet (jamonoy taw) ak ju noor (jamonoy fendi).")

# Check the type of the output
type(doc_wo)

stanza.models.common.doc.Document

This returns a Stanza [*Document*](https://stanfordnlp.github.io/stanza/data_objects.html#document) object, which contains the linguistic annotations created by passing the text through the pipeline.

To begin with, the attribute `sentences` of a *Document* object contains a list of lists, in which each list stands for a sentence.

Let's use the brackets to access the first item `[0]` in the list.

In [6]:
# Get the first item in the list of sentences
doc_wo.sentences[0]

[
  {
    "id": 1,
    "text": "Réew",
    "lemma": "réew",
    "upos": "NOUN",
    "xpos": "NOUN",
    "head": 4,
    "deprel": "nsubj",
    "misc": "start_char=0|end_char=4"
  },
  {
    "id": 2,
    "text": "maa",
    "lemma": "a",
    "upos": "AUX",
    "xpos": "AUX",
    "feats": "PronType=Prs",
    "head": 4,
    "deprel": "aux",
    "misc": "start_char=5|end_char=8"
  },
  {
    "id": 3,
    "text": "ngi",
    "lemma": "ngi",
    "upos": "AUX",
    "xpos": "AUX",
    "feats": "Aspect=Prog",
    "head": 4,
    "deprel": "aux",
    "misc": "start_char=9|end_char=12"
  },
  {
    "id": 4,
    "text": "lebe",
    "lemma": "lebe",
    "upos": "VERB",
    "xpos": "VERB",
    "feats": "Mood=Ind|VerbForm=Fin",
    "head": 0,
    "deprel": "root",
    "misc": "start_char=13|end_char=17"
  },
  {
    "id": 5,
    "text": "turam",
    "lemma": "tur",
    "upos": "NOUN",
    "xpos": "NOUN",
    "feats": "Number=Sing|Poss=Yes",
    "head": 4,
    "deprel": "obj",
    "misc": "start_char=18|e

Although the output contains both brackets `[]` and curly braces `{}`, which Python typically uses for lists and dictionaries, respectively, the output is not a list with nested dictionaries, but a Stanza [*Sentence*](https://stanfordnlp.github.io/stanza/data_objects.html#sentence) object.

In [7]:
# Check the type of the first item in the Document object
type(doc_wo.sentences[0])

stanza.models.common.doc.Sentence

The *Sentence* object contains various attributes and methods for accessing the linguistic annotations created by the language model.

If we wish to interact with the annotations using data structures native to Python, we can use the `to_dict()` method to cast the annotations into a list of dictionaries, where each dictionary stands for a single Stanza [*Token*](https://stanfordnlp.github.io/stanza/data_objects.html#token) object.

The *key* and *value* pairs in these dictionaries contain the linguistic annotations for each *Token*.

In [8]:
# Cast the first Sentence object into a Python dictionary; store under variable 'doc_dict'
doc_dict = doc_wo.sentences[0].to_dict()

# Get the dictionary for the first Token
doc_dict[0]

{'id': 1,
 'text': 'Réew',
 'lemma': 'réew',
 'upos': 'NOUN',
 'xpos': 'NOUN',
 'head': 4,
 'deprel': 'nsubj',
 'misc': 'start_char=0|end_char=4'}

As you can see, the dictionary consists of keys and values, which hold the linguistic annotations.

We can retrieve a list of keys available for a Python dictionary using the `keys()` method.

In [9]:
# Get a list of keys for the first Token in the dictionary 'doc_dict'
doc_dict[0].keys()

dict_keys(['id', 'text', 'lemma', 'upos', 'xpos', 'head', 'deprel', 'misc'])

Let's retrieve the value under the key `lemma`.

In [10]:
# Get the value under key 'lemma' for the first item [0] in the dictionary 'doc_dict'
doc_dict[0]['lemma']

'réew'

This returns the lemma of the word "réew", which stands for "country".

### Processing multiple texts using Stanza

To process multiple documents with Stanza, first collect the documents as string objects into a Python list.

Let's define a toy example with a couple of example documents in Wolof and store them under the variable `str_docs`.

In [11]:
# Define a Python list consisting of two strings
str_docs = ['Lislaam a ngi njëkk a tàbbi ci Senegaal ci diggante VIIIeelu xarnu ak IXeelu xarnu, ña ko fa dugal di ay yaxantukat yu araab-yu-berber.',
            'Li ëpp ci gëstu yi ñu def ci wàllug Gëstu-askan (walla demogaraafi) ci Senegaal dafa sukkandiku ci Waññ (recensement) yi ñu jotoon a def ci 1976, 1988 rawati na 2002.'] 

Next, we create a list of Stanza *Document* objects using a Python list comprehension.

These *Document* objects are annotated for their linguistic features when they are passed through a *Pipeline*.

At this stage, we simply cast each string in the list `str_docs` to a Stanza *Document* object.

Before proceeding to create the *Document* objects, let's examine the list comprehension in greater detail.

Just like lists, list comprehensions are surrounded by brackets `[]`.

Next, on the right-hand side of the `for` statement, we use the variable `doc` to refer to items in the list `str_docs` that we are looping over.

Now that we can refer to list items using the variable `doc`, we can define what we do to each item on the left-hand side of the `for` statement.

For each item in the list `str_docs`, we initialise an empty `Document` object and pass two inputs to the newly created object: 

 1. an empty list `[]` that will be populated with linguistic annotations, 
 2. the contents of the string variable under `doc` to the argument `text`. 

In [12]:
# Use a list comprehension to create a Python list with Stanza Document objects.
docs_wo_in = [stanza.Document([], text=doc) for doc in str_docs]

# Call the variable to check the output
docs_wo_in

[[], []]

Don't let the output fool you: what looks like two empty Python lists nested within another list are actually Stanza *Document* objects.

In [13]:
docs_wo_out = nlp_wo(docs_wo_in)

docs_wo_out

[[
   [
     {
       "id": 1,
       "text": "Lislaam",
       "lemma": "Lislaam",
       "upos": "PROPN",
       "xpos": "NAME",
       "head": 4,
       "deprel": "nsubj",
       "misc": "start_char=0|end_char=7"
     },
     {
       "id": 2,
       "text": "a",
       "lemma": "a",
       "upos": "AUX",
       "xpos": "AUX",
       "feats": "PronType=Prs",
       "head": 4,
       "deprel": "aux",
       "misc": "start_char=8|end_char=9"
     },
     {
       "id": 3,
       "text": "ngi",
       "lemma": "ngi",
       "upos": "AUX",
       "xpos": "AUX",
       "feats": "Aspect=Prog",
       "head": 4,
       "deprel": "aux",
       "misc": "start_char=10|end_char=13"
     },
     {
       "id": 4,
       "text": "njëkk",
       "lemma": "njëkk",
       "upos": "VERB",
       "xpos": "VERB",
       "feats": "Mood=Ind|VerbForm=Fin",
       "head": 0,
       "deprel": "root",
       "misc": "start_char=14|end_char=19"
     },
     {
       "id": 5,
       "text": "a",
       "lemma

## Interfacing Stanza with spaCy

If you are more familiar with the spaCy library for natural language processing, you can also use some of the Stanza language models in spaCy!

This can be achieved using a Python library named [spacy-stanza](https://spacy.io/universe/project/spacy-stanza), which interfaces the two libraries.

Given that Stanza currently has more pre-trained language models available than spaCy, the spacy-stanza library considerably increases the number of languages available for spaCy.

There is, however, **one major limitation**: the language model in Stanza must be [supported by spaCy](https://spacy.io/usage/models#languages).

For example, we cannot use the Stanza language model for Wolof in spaCy, because spaCy does not support the Wolof language.

To start using Stanza language models in spaCy, let's start by importing the spacy-stanza library (`spacy_stanza`).

In [14]:
# Import the spaCy and spacy-stanza libraries
import spacy
import spacy_stanza

Now that we have imported both spaCy and spacy-stanza libraries into Python, let's download a Stanza language model for Finnish into the same directory where we placed the language model for Wolof above.

In [15]:
# Download a Stanza language model for Finnish into the directory "../stanza_models"
stanza.download(lang="fi", model_dir="../stanza_models")

2021-03-08 20:17:25 INFO: Downloading default packages for language: fi (Finnish)...
2021-03-08 20:17:26 INFO: File exists: ../stanza_models/fi/default.zip.
2021-03-08 20:17:27 INFO: Finished downloading models and saved to ../stanza_models.


Because spaCy supports [the Finnish language](https://spacy.io/usage/models#languages), we can load the Stanza language model for Finnish into spaCy using the spacy-stanza library.

This can be achieved using the `load_pipeline()` function available in spacy-stanza.

This function takes two arguments: the argument `name` requires a string object that defines the two-letter code for the Stanza language model to be loaded, while `dir` takes a string object that points to the directory with Stanza language models.

In [16]:
# Use the load_pipeline function to load a Stanza model into spaCy.
# Assign the result under the variable 'nlp'.
nlp_fi = spacy_stanza.load_pipeline(name="fi", dir="../stanza_models")

2021-03-08 20:17:32 INFO: Loading these models for language: fi (Finnish):
| Processor | Package |
-----------------------
| tokenize  | tdt     |
| mwt       | tdt     |
| pos       | tdt     |
| lemma     | tdt     |
| depparse  | tdt     |

2021-03-08 20:17:32 INFO: Use device: cpu
2021-03-08 20:17:32 INFO: Loading: tokenize
2021-03-08 20:17:32 INFO: Loading: mwt
2021-03-08 20:17:32 INFO: Loading: pos
2021-03-08 20:17:32 INFO: Loading: lemma
2021-03-08 20:17:32 INFO: Loading: depparse
2021-03-08 20:17:33 INFO: Done loading processors!


This will produce the same kind of output as loading a Stanza model.

If we examine the resulting object under the variable `nlp_fi` using Python's `type()` function, we will see that the object is indeed a spaCy *Language* object. 

In [17]:
# Check the type of the object under 'nlp_fi'
type(nlp_fi)

spacy.lang.fi.Finnish

Generally, this object behaves just like any other spaCy *Language* object that we learned to use in [Part II](../part_ii/03_basic_nlp.ipynb#Performing-basic-NLP-tasks-using-spaCy).

We can explore the use of this model by processing a few sentences from a recent [news article](https://yle.fi/aihe/artikkeli/2021/03/08/yleiso-aanesti-tarja-halonen-on-inspiroivin-nainen-karkikolmikkoon-ylsivat-myos) in written Finnish.

We feed the text as a string object to the *Language* object under `nlp_fi` and store the result under the variable `doc_fi`.

In [18]:
# Feed the text to the language model under 'nlp_fi', store result under 'doc_fi'
doc_fi = nlp_fi('Tove Jansson keräsi 148 ääntä eli 18,2% annetuista äänistä. Kirjailija, kuvataiteilija ja pilapiirtäjä tuli kansainvälisesti tunnetuksi satukirjoistaan ja sarjakuvistaan.')

Let's continue by retrieve a list of sentences from the *Doc* object, which are available under the attribute `sents`.

Remember that the object under the `sents` attribute is a Python generator that yields *Doc* objects. To examine them, we must catch the objects into a suitable data structure, which in this case is a list.

In [19]:
# Get sentences contained in the Doc object 'doc_fi'.
# Cast the result into list.
sents_fi = list(doc_fi.sents)

# Call the variable to check the output
sents_fi

[Tove Jansson keräsi 148 ääntä eli 18,2% annetuista äänistä.,
 Kirjailija, kuvataiteilija ja pilapiirtäjä tuli kansainvälisesti tunnetuksi satukirjoistaan ja sarjakuvistaan.]

We can, for example, to use the `render()` function in the `displacy` submodule of spaCy to visualise the syntactic dependencies for the first sentence under `sents_fi`.

In [20]:
# Import the displacy submodule
from spacy import displacy

# Use the render function to render the first item [0] in the list 'sents_fi'.
# Pass the argument 'style' with the value 'dep' to visualise syntactic dependencies.
displacy.render(sents_fi[0], style='dep')

[W109] Unable to save user hooks while serializing the doc. Re-add any required user hooks to the doc after processing.


Note that spaCy will raise a warning about storing custom attributes when writing the *Doc* object to disk for visualisation. 

In [ ]:
for token in doc_fi:
    
    print(token, token.lemma_, token.dep_)

In [ ]:
list(doc_fi.sents)

In [ ]:
doc = nlp_fi("kansainvälisesti")